### Question 1

In [2]:
import time
from collections import deque
from random import choice

IP_POOL = ['192.168.0.1', '192.168.0.2', '192.168.0.3']

In [3]:
def simulate_http_request(ip):
    print(f"Request made from IP: {ip}")


class Throttler:
    def __init__(self, rate_limit):
        self.requests = deque()
        self.rate_limit = rate_limit

    def allow_request(self):
        current_time = time.time()
        # Remove timestamps older than 1 minute
        while self.requests and self.requests[0] < current_time - 60:
            self.requests.popleft()

        if len(self.requests) < self.rate_limit:
            return True
        else:
            return False

In [4]:
def make_requests(n):
    throttler = Throttler(5)
    ip_index = 0

    for _ in range(n):
        while not throttler.allow_request():
            print("Request throttled. Too many requests. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        ip = IP_POOL[ip_index % len(IP_POOL)]
        simulate_http_request(ip)
        throttler.requests.append(time.time())
        ip_index += 1

In [7]:
make_requests(10)

#Force Stopped it myself (hence the keyboard interrupt)

Request made from IP: 192.168.0.1
Request made from IP: 192.168.0.2
Request made from IP: 192.168.0.3
Request made from IP: 192.168.0.1
Request made from IP: 192.168.0.2
Request throttled. Too many requests. Waiting...
Request throttled. Too many requests. Waiting...
Request throttled. Too many requests. Waiting...
Request throttled. Too many requests. Waiting...


KeyboardInterrupt: 

### Question 2

We can use LSH to extract common image features and run on only the most defining features.
Additionally we can grayscale or rescale the images to reduce their size(dimension) for more accurate information with less noise.

The Kernel trciks that come to mind are the Radial Basis funtions.
Also vectorizing the entire dataset by feature scaling is also a possible with a lot of a lot preprocessing for better accuracy/

In [8]:
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import os
from sklearn.model_selection import train_test_split

In [9]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            labels.append(0 if 'cat' in filename else 1)
    return images, labels

In [10]:
def preprocess_images(images, target_size=(64, 64)):
    processed_images = []
    for img in images:
        print(img.flatten().shape)
        img_resized = resize(img, target_size, anti_aliasing=True, mode='reflect')
        processed_images.append(img_resized.flatten())
    return np.array(processed_images)

In [12]:
folder = 'train/test_train'
images, labels = load_images_from_folder(folder)
processed_images = preprocess_images(images)

(561000,)
(252000,)
(732033,)
(603291,)
(177600,)
(359121,)
(579000,)
(302841,)
(112230,)
(448800,)
(523950,)
(497004,)
(561000,)
(530130,)
(561000,)
(319332,)
(561000,)
(377916,)
(274050,)
(187110,)
(561375,)
(417690,)
(561000,)
(377610,)
(201600,)
(561000,)
(327609,)
(556884,)
(281520,)
(561375,)
(718560,)
(372708,)
(290520,)
(561000,)
(504000,)
(471555,)
(633000,)
(561375,)
(502992,)
(561000,)
(561000,)
(658080,)
(556884,)
(585000,)
(265920,)
(453591,)
(256320,)
(200340,)
(561000,)
(551286,)
(561000,)
(561375,)
(458880,)
(28560,)
(358800,)
(229440,)
(703590,)
(428895,)
(561375,)
(561375,)
(526944,)
(377244,)
(133980,)
(422154,)
(561000,)
(186750,)
(715500,)
(666165,)
(346752,)
(561000,)
(687123,)
(409374,)
(561375,)
(561375,)
(242550,)
(387600,)
(624249,)
(188250,)
(561000,)
(561375,)
(482034,)
(262080,)
(112569,)
(739500,)
(643500,)
(561000,)
(214080,)
(370656,)
(561000,)
(645207,)
(517500,)
(256512,)
(561000,)
(517500,)
(561000,)
(531690,)
(231660,)
(561375,)
(621000,)
(274314,)
(

In [13]:
features_train, features_test, labels_train, labels_test = train_test_split(
    processed_images, labels, test_size=0.2, random_state=42)

In [20]:
def train_svm(features, labels, epochs=600, learning_rate=0.03, lambda_param=0.1):
    num_samples, num_features = features.shape
    weights = np.zeros(num_features)  # Initialize weight vector
    for epoch in range(epochs):
        for i in range(num_samples):
            if labels[i] == 0:
                labels[i] = -1  # Convert labels to -1 and 1 for SVM
            # Compute the margin and update weights using gradient descent
            margin = labels[i] * np.dot(features[i], weights)
            if margin < 1:
                weights = weights - learning_rate * (2 * lambda_param * weights - labels[i] * features[i])
    return weights

In [21]:
def evaluate_svm(weights, features, labels):
    num_samples = features.shape[0]
    num_correct = 0
    for i in range(num_samples):
        prediction = np.sign(np.dot(features[i], weights))
        if prediction == labels[i]:
            num_correct += 1
    accuracy = num_correct / num_samples
    return accuracy

In [22]:
weights = train_svm(features_train, labels_train)
accuracy = evaluate_svm(weights, features_test, labels_test)
print("Accuracy of the SVM classifier: {:.2f}%".format(accuracy * 100))

Accuracy of the SVM classifier: 50.56%
